In [22]:
import os
import logging
from clickhouse_driver import Client
from dotenv import load_dotenv

# Загрузка переменных окружения
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("adjust_status.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Глобальные переменные
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Подключение к ClickHouse
client = Client(
    host=os.getenv('CLICKHOUSE_HOST', '10.95.19.132'),
    user=os.getenv('CLICKHOUSE_USER', 'default'),
    password=os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi'),
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)


def adjust_status():
    """
    Корректирует поле Status в таблице OlapCube_VNV на основе значения balance_total и статусов в Status_P.
    """
    try:
        # Извлечь первую дату
        query_first_date = f"""
        SELECT MIN(Dates) AS first_date
        FROM {database_name}.OlapCube_VNV
        """
        first_date_result = client.execute(query_first_date)
        first_date = first_date_result[0][0]
        logger.info(f"Первая дата в OlapCube_VNV: {first_date}")

        # Получить значение balance_total на первую дату
        query_balance_total = f"""
        SELECT balance_total
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{first_date}'
        LIMIT 1
        """
        balance_total = client.execute(query_balance_total)[0][0]
        logger.info(f"balance_total на первую дату: {balance_total}")

        # Извлечь записи Status_P на первую дату
        query_status_p = f"""
        SELECT serialno, Status_P
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{first_date}'
        ORDER BY serialno
        """
        status_p_data = client.execute(query_status_p)
        logger.info(f"Извлечено {len(status_p_data)} записей на первую дату.")

        # Разделение записей по статусам
        exploitation = [row for row in status_p_data if row[1] == "Эксплуатация"]
        repaired = [row for row in status_p_data if row[1] == "Исправен"]
        inactive = [row for row in status_p_data if row[1] == "Неактивно"]

        logger.info(f"Начальное количество 'Эксплуатация': {len(exploitation)}")
        logger.info(f"Начальное количество 'Исправен': {len(repaired)}")
        logger.info(f"Начальное количество 'Неактивно': {len(inactive)}")

        changed_serialnos = []
        if balance_total == 0:
            # Копируем все значения Status_P в Status
            update_query = f"""
            ALTER TABLE {database_name}.OlapCube_VNV 
            UPDATE Status = Status_P
            WHERE Dates = '{first_date}'
            """
            client.execute(update_query)
            logger.info("Статусы скопированы без изменений.")
        elif balance_total > 0:
            # Меняем первых balance_total записей "Эксплуатация" на "Исправен"
            rows_to_update = exploitation[:int(balance_total)]
            for row in rows_to_update:
                update_query = f"""
                ALTER TABLE {database_name}.OlapCube_VNV 
                UPDATE Status = 'Исправен'
                WHERE serialno = '{row[0]}' AND Dates = '{first_date}'
                """
                client.execute(update_query)
                changed_serialnos.append(row[0])
            logger.info(f"Изменено {len(changed_serialnos)} записей: 'Эксплуатация' -> 'Исправен'. Serialno: {', '.join(changed_serialnos)}")
        elif balance_total < 0:
            # Меняем abs(balance_total) записей "Исправен" или "Неактивно" на "Эксплуатация"
            deficit_count = abs(int(balance_total))
            rows_to_add = repaired[:deficit_count]
            deficit_count -= len(rows_to_add)
            rows_to_add.extend(inactive[:deficit_count])
            for row in rows_to_add:
                update_query = f"""
                ALTER TABLE {database_name}.OlapCube_VNV 
                UPDATE Status = 'Эксплуатация'
                WHERE serialno = '{row[0]}' AND Dates = '{first_date}'
                """
                client.execute(update_query)
                changed_serialnos.append(row[0])
            logger.info(f"Изменено {len(changed_serialnos)} записей: '{row[1]}' -> 'Эксплуатация'. Serialno: {', '.join(changed_serialnos)}")

        # Копирование оставшихся значений из Status_P в Status
        update_remaining_query = f"""
        ALTER TABLE {database_name}.OlapCube_VNV
        UPDATE Status = Status_P
        WHERE Dates = '{first_date}' AND Status IS NULL
        """
        client.execute(update_remaining_query)
        logger.info("Оставшиеся значения скопированы из Status_P в Status.")

    except Exception as e:
        logger.error(f"Ошибка при корректировке статусов: {e}", exc_info=True)


def main():
    adjust_status()


if __name__ == "__main__":
    main()


2024-12-04 12:22:19,298 - INFO - Первая дата в OlapCube_VNV: 2024-11-25
2024-12-04 12:22:19,309 - INFO - balance_total на первую дату: 6.0
2024-12-04 12:22:19,343 - INFO - Извлечено 420 записей на первую дату.
2024-12-04 12:22:19,344 - INFO - Начальное количество 'Эксплуатация': 162
2024-12-04 12:22:19,345 - INFO - Начальное количество 'Исправен': 9
2024-12-04 12:22:19,346 - INFO - Начальное количество 'Неактивно': 235
2024-12-04 12:22:19,411 - INFO - Изменено 6 записей: 'Эксплуатация' -> 'Исправен'. Serialno: 038159, 038189, 0828С46, 0828С47, 0918С36, 0938С112
2024-12-04 12:22:19,418 - INFO - Оставшиеся значения скопированы из Status_P в Status.
